In [ ]:
import chromadb
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch
from sentence_transformers import SentenceTransformer
from langchain_huggingface import HuggingFacePipeline, ChatHuggingFace

# Fail 1

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

In [ ]:
def load_pdf(file_path):
    loader = PyPDFLoader(file_path)
    pages = []
    for page in loader.lazy_load():
        pages.append(page)
    return pages

In [ ]:
import os
import multiprocessing

files_path = "data/files"
pdf_files = [f for f in os.listdir(files_path) if f.endswith('.pdf')]
pdf_extracted_pages = []

with multiprocessing.Pool(processes=20) as pool:
        results = pool.map(load_pdf, [files_path + "/"+ x for x in pdf_files])

In [ ]:
print(len(results))

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
chunked_documents = []

text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=512,
    chunk_overlap=100,
    length_function=len,
    is_separator_regex=False,
)

def chunk_files(pdf_extracted_pages):
    split_documents = []
    split_documents.extend(text_splitter.split_documents(pdf_extracted_pages))
    return split_documents

# print(chunk_files(results[0]))
    
with multiprocessing.Pool(processes=20) as pool:
    chunked_documents = pool.map(chunk_files, results)

In [ ]:
print(len(chunked_documents))

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_core.prompt_values import PromptValue
class Extractor:
    def __init__(self, chat_model):
        self.chat_model=chat_model
        self.KEYWORD_EXTRACTOR_PROMPT = "{context_str}. Generate {num_keywords} keywords from the text. The general theme is X-linked dominant rare diseases. Format as comma separated. Keywords:"
        
        self.TITLE_EXTRACTOR_PROMPT = """\
    Context: {context_str}. Give a title that summarizes all of \
    the unique entities, titles or themes found in the context. Title: """
        
        self.QA_EXTRACTOR_PROMPT = """\
        Here is the context:
        {context_str}
        
        Given the contextual information, \
        generate {num_questions} questions this context can provide \
        specific answers to which are unlikely to be found elsewhere.
        
        Higher-level summaries of surrounding context may be provided \
        as well. Try using these summaries to generate better questions \
        that this context can answer.
        
        """



    def KeywordExtractor(self, document, num_keywords):
        context_str = document.page_content
        prompt_string = self.KEYWORD_EXTRACTOR_PROMPT.format(context_str=context_str, num_keywords=num_keywords)
        # prompt = PromptTemplate.from_template(self.KEYWORD_EXTRACTOR_PROMPT)
        # prompt.format(context_str=context_str, num_keywords=num_keywords)
        prompt = PromptValue.to_messages(prompt_string)
        print(self.chat_model.invoke(prompt_string))
        
        
        
    def TitleExtractor(self, document):
        pass
        
    def QuestionAnswerExtractor(self,document, num_questions):
        pass
            


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.1-8B-Instruct")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.1-8B-Instruct", 
                                             pad_token_id=tokenizer.eos_token_id,
                                             device_map = "auto", 
                                             trust_remote_code=True
                                            )

pipe = pipeline(
    "text-generation", model=model, tokenizer=tokenizer, max_new_tokens=120)
hf = HuggingFacePipeline(pipeline=pipe)
chat_model = ChatHuggingFace(llm=hf)
extractor = Extractor(chat_model)
print(extractor.KeywordExtractor(chunked_documents[0][0],2))

# Attempt 2

In [1]:
from langchain_community.document_loaders import PyPDFLoader
import os
import multiprocessing

In [2]:
def load_pdf(file_path):
    loader = PyPDFLoader(file_path)
    pages = []
    for page in loader.lazy_load():
        pages.append(page)
    return pages

In [3]:
files_path = "data/files"
pdf_files = [f for f in os.listdir(files_path) if f.endswith('.pdf')]
pdf_extracted_pages = []

with multiprocessing.Pool(processes=20) as pool:
        results = pool.map(load_pdf, [files_path + "/"+ x for x in pdf_files])

Ignoring wrong pointing object 13 0 (offset 0)
incorrect startxref pointer(2)
Ignoring wrong pointing object 12 0 (offset 0)
Ignoring wrong pointing object 43 0 (offset 0)
Ignoring wrong pointing object 55 0 (offset 0)
Ignoring wrong pointing object 12 0 (offset 0)
Ignoring wrong pointing object 14 0 (offset 0)
Ignoring wrong pointing object 13 0 (offset 0)
Ignoring wrong pointing object 10 0 (offset 0)
Ignoring wrong pointing object 12 0 (offset 0)
Ignoring wrong pointing object 14 0 (offset 0)
Ignoring wrong pointing object 18 0 (offset 0)
Ignoring wrong pointing object 92 0 (offset 0)
Multiple definitions in dictionary at byte 0x84407 for key /Ascent
Ignoring wrong pointing object 12 0 (offset 0)
Ignoring wrong pointing object 14 0 (offset 0)
Ignoring wrong pointing object 18 0 (offset 0)
Ignoring wrong pointing object 20 0 (offset 0)
Ignoring wrong pointing object 35 0 (offset 0)
Ignoring wrong pointing object 64 0 (offset 0)


In [4]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
chunked_documents = []

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=512,
    chunk_overlap=100,
    length_function=len,
    is_separator_regex=False,
)

def chunk_files(pdf_extracted_pages):
    split_documents = []
    split_documents.extend(text_splitter.split_documents(pdf_extracted_pages))
    return split_documents

    
with multiprocessing.Pool(processes=20) as pool:
    chunked_documents = pool.map(chunk_files, results)

In [6]:
c = 0
for document in chunked_documents:
    print(len(document))

45
11
76
69
31
106
139
83
42
138
92
71
38
23
94
76
75
72
142
49
169
105
17
105
162
99
67
88
41
111
156
48
209
125
137
64
53
143
88
14
60
84
118
121
157
35
26
50
25
121
53
197
117
72
151
63
196
40
179
64
192
309
96
159
28
29
34
109
38
149
69
52
45
18
39
155
48
93
23
66
50
73
25
48
70
71
39
101
229
39
31
246
139
86
52
113
116
32
40
70
54
53
47
137
150
156
42
47
85
42
46
41
179
73
23
84
106
24
45
265
74
141
73
127
20
15
32
36
41
100
50
116
107
112
160
56
238
53
134
102
338
88
286
88
57
152
56
101
125
106
142
44
79
78
126
170
28
60
95
195
79
83
370
45
130
131
142
93
139
89
166
213
228
174
55
104
47
72
81
70
26
27
34
58
65
128
61
76
40
89
73
130
94
138
23
48
163
97
65
29
73
64
42
74
61
12
192
130
37
0
54
20
143
125
230
24
98
172
64
175
69
12
70
56
76
81
9
208
29
32
31
80
66
164
26
122
139
52
251
71
14
88
104
104
62
366
22
48
124
76
43
54
59
51
109
38
349
31
58
116
93
148
30
234
39
50
83
71
52
228
40
123
28
95
49
136
135
148
101
165
36
145
65
180
111
106
49
176
37
250
374
125
102
8
111
137
3

In [ ]:
OPENAI_BATCH_SIZE = 50000

for i in range(0, len(nodes), OPENAI_BATCH_SIZE):
    